In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
import cv2
import matplotlib.pyplot as mp

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/MLassign/train'
train_data = os.listdir(path)
# imageList.remove("labels.txt")
with open("/content/drive/MyDrive/MLassign/labels.txt", 'r') as fl:
  lb = fl.readlines()
lb = [line.rstrip('\n') for line in lb]

Xtrain = []
ytrain = []

Xtest = []
ytest = []

split_ratio = 0.8

count = len(train_data)*split_ratio
i = 0 
for image in train_data:
  if(i<count):
    #  add to train list
    Xtrain.append(image)
    index = int(image.split(".")[0])
    ytrain.append(lb[index])
  else:
    Xtest.append(image)
    index = int(image.split(".")[0])
    ytest.append(lb[index])
  i += 1

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tqdm import tqdm

In [ ]:

labels = {'ALPHA' : 0,
 'BETA' : 1,
 'CHI' : 2,
 'DELTA' : 3,
 'EPSILON': 4,
 'ETA' : 5,
 'GAMMA' : 6,
 'IOTA' : 7,
 'KAPPA' : 8,
 'LAMDA': 9,
 'MU' :10,
 'NU' : 11,
 'OMEGA' : 12,
 'OMICRON':13,
 'PHI' : 14,
 'PI' : 15,
 'PSI' : 16,
 'RHO' : 17,
 'SIGMA' : 18,
 'TAU' : 19,
 'THETA' : 20,
 'UPSILON' : 21,
 'XI' : 22,
 'ZETA': 23}

In [ ]:
#def backgr(im,im_freq):
  #backg=np.where((im[:,:,0]==im_freq[0]) & (im[:,:,1]==im_freq[1]) & (im[:,:,2]==im_freq[2]))
  #im[backg]=np.array([255,255,255],dtype=np.unit8)
  #return im


In [ ]:
def model():
  for imageIndex in tqdm(range(len(Xtrain))):
    img = cv2.imread(os.path.join(path, Xtrain[imageIndex]))
    # Find pixel with max freq
    img_temp = img.copy()
    unique, counts = np.unique(img_temp.reshape(-1, 3), axis=0, return_counts=True)
    img_freq = unique[np.argmax(counts)]
    y_shape, x_shape = img.shape[:2]

    # change the background with white
    backg = np.where((img[:,:,0]==img_freq[0]) & (img[:,:,1]==img_freq[1]) & (img[:,:,2]==img_freq[2]))
    img[backg] = np.array([255,255,255], dtype= np.uint8)
    # Now Dilate the image 
    kernel = np.ones((5, 5), np.uint8)
    img_dilat = cv2.dilate(img, kernel, iterations=1)
    gray = cv2.cvtColor(img_dilat, cv2.COLOR_BGR2GRAY)
    # Segment image into 3 character
    start = True
    cPrev = 0
    list1 = []
    for i in range(500):
      vpp = np.sum(gray[:,i] < 250)
      if vpp > 12:
        if start:
          # print(i)
          temp = i
          start = False
        cPrev += 1
      if vpp == 0:
        if start == False:
          # print(i)
          if cPrev > 30:
            sizediff = i-temp
            if (sizediff < 150):
              x_t = temp - int((150 - sizediff)/2)
              t = x_t if x_t > 0 else 0
              t= t if t + 150 < 500 else 350
              y = 0
            else:
              t = i
              y = 0
            list1.append((t , y, 150, 150))
            cPrev = 0
          start = True    
    list1size = len(list1)
    if list1size != 3 :
      list1 = [(15 , 0 , 150, 150),(175, 0, 150, 150),(335, 0, 150, 150)]
      # print(mylist)
      # print(imageFile)
      # print(listSize)
      # print(mylist)
      # print(vpp)
      # return "failed"

    # Got the segment of the image
    labellist1 = ytrain[imageIndex].split(",")
    numberLabel = []
    for la in labellist1:
        numberLabel.append(labels[la])
    for boxIndex in range(3):
      t, y = list1[boxIndex][0], list1[boxIndex][1]
      finalImage = gray[y:y + 150, t:t + 150]
      finalImage = cv2.resize(finalImage, (30, 30))
      finalImage = finalImage.flatten()
      d_xtrain.append(finalImage)
      # get the labels
      d_ytrain.append(numberLabel[boxIndex])
  return "success"
d_xtrain = []
d_ytrain = []
model()

100%|██████████| 1600/1600 [01:59<00:00, 13.34it/s]


'success'

In [ ]:
def predict():
  for imageIndex in tqdm(range(len(Xtrain))):
    img = cv2.imread(os.path.join(path, Xtrain[imageIndex]))
    # Find pixel with max freq
    img_temp = img.copy()
    unique, counts = np.unique(img_temp.reshape(-1, 3), axis=0, return_counts=True)
    img_freq = unique[np.argmax(counts)]
    y_shape, x_shape = img.shape[:2]

    # change the background with white
    backg = np.where((img[:,:,0]==img_freq[0]) & (img[:,:,1]==img_freq[1]) & (img[:,:,2]==img_freq[2]))
    img[backg] = np.array([255,255,255], dtype= np.uint8)
    # Now Dilate the image 
    kernel = np.ones((5, 5), np.uint8)
    img_dilat = cv2.dilate(img, kernel, iterations=1)
    gray = cv2.cvtColor(img_dilat, cv2.COLOR_BGR2GRAY)
    # Segment image into 3 character
    start = True
    cPrev = 0
    list1 = []
    for i in range(500):
      vpp = np.sum(gray[:,i] < 250)
      if vpp > 12:
        if start:
          # print(i)
          temp = i
          start = False
        cPrev += 1
      if vpp == 0:
        if start == False:
          # print(i)
          if cPrev > 30:
            sizediff = i-temp
            if (sizediff < 150):
              x_t = temp - int((150 - sizediff)/2)
              t = x_t if x_t > 0 else 0
              t = t if t + 150 < 500 else 350
              y = 0
            else:
              t = i
              y = 0
            list1.append((t , y, 150, 150))
            cPrev = 0
          start = True    
    list1size = len(list1)
    if list1size != 3 :
      list1 = [(15 , 0 , 150, 150),(175, 0, 150, 150),(335, 0, 150, 150)]
      # print(mylist)
      # print(imageFile)
      # print(listSize)
      # print(mylist)
      # print(vpp)
      # return "failed"

    # Got the segment of the image
    labellist1 = ytrain[imageIndex].split(",")
    numberLabel = []
    for la in labellist1:
        numberLabel.append(labels[la])
    for boxIndex in range(3):
      t, y = list1[boxIndex][0], list1[boxIndex][1]
      finalImage = gray[y:y + 150, t:t + 150]
      finalImage = cv2.resize(finalImage, (30, 30))
      finalImage = finalImage.flatten()
      dfXtest.append(finalImage)
      # get the labels
      dfytest.append(numberLabel[boxIndex])
  return "success"

dfXtest = []
dfytest = []
print(predict())

100%|██████████| 1600/1600 [01:56<00:00, 13.78it/s]

success


In [ ]:
from sklearn.svm import SVC
import pickle
model = SVC(kernel = 'linear', C = 1).fit(dfXtest,dfytest)
predictions = model.predict(dfXtest)
count = 0
for i in range(len(predictions)):
    if predictions[i] == dfytest[i]:
        count += 1
print(count/(i+1)*100)


filename = '/content/drive/MyDrive/MLassign/modelSVM.sav'
pickle.dump(model, open(filename, 'wb'))

100.0
